In [ ]:
### 위키피디아 wikicorpus 이용하여 model만들기 ###
# 2017-07-29

# 참고사이트
# http://blog.theeluwin.kr/post/146591096133/%ED%95%9C%EA%B5%AD%EC%96%B4-word2vec
# http://textminingonline.com/training-word2vec-model-on-english-wikipedia-by-gensim

import gensim
from gensim.corpora import WikiCorpus, wikicorpus
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

# 1. corpus(말뭉치)를 만든다
# 한 문장이 한줄에 들어가야되는데 wikiCorpus를 이용하면 한 단락이 리스트에 들어간다. 잘모르겠다.. 그냥해도 나오긴 하는 듯

wiki = WikiCorpus('kowiki-latest-pages-articles.xml.bz2', dictionary={}) # multistream 파일 안읽힘

In [ ]:
# 2. corpus를 txt파일로 저장한다. 시간 좀 걸림..

import six

# type(wiki)
i = 1
with open('result.txt', 'w', encoding='utf-8') as f:
    for text in wiki.get_texts():
        if six.PY3:
            f.write(' '.join(text) + '\n')
        #   ###another method###
        #    output.write(
        #            space.join(map(lambda x:x.decode("utf-8"), text)) + '\n')
        else:
            f.write(space.join(text) + "\n")
        i = i + 1
        if (i % 10000 == 0):
            print(i)



# dic 형식이나 matrix market 형식의 파일로 저장할 수 있다.
# wiki.dictionary.save("wiki_dict.dict")
# MmCorpus.serialize("wiki_corpus.mm", wiki)

In [ ]:
# 3. 조사가 다 붙어있기 때문에 twitter 이용해서 떼준다.
# 제임스/Noun 지미/Noun 카터/Noun... 이런식으로 / 여기도 시간 좀 걸림

from konlpy.tag import Twitter

twitter = Twitter()

def flat(content):
    return ["{}/{}".format(word, tag) for word, tag in twitter.pos(content)]

k = open('result_k.txt','w')
with open('result.txt','r') as f:
    for line in f.readlines():
        k.write(' '.join(flat(line)) + '\n')

k.close()

In [ ]:
# 4. 학습시켜서 model 만들고 파일로 저장 
# 설정 아직 다 모르겠음.. 빠르게 학습하기 위해서 설정했는데 공부 더 필요!! / 한시간정도 걸림

import multiprocessing
from gensim.models.word2vec import LineSentence

config = {
    'min_count': 50,  # 등장 횟수가 5 이하인 단어는 무시
    'size': 300,  # 300차원짜리 벡터스페이스에 embedding
    'sg': 0,  # 0이면 CBOW, 1이면 skip-gram을 사용한다
    'batch_words': 10000,  # 사전을 구축할때 한번에 읽을 단어 수
    'iter': 10,  # 보통 딥러닝에서 말하는 epoch과 비슷한, 반복 횟수
    'workers': multiprocessing.cpu_count(),
}

model = gensim.models.Word2Vec(LineSentence('result_k.txt'),**config) # LineSentence 이용하여 한 줄(리스트)씩 읽어올 수 있는 듯
model.save('result_k.model')

In [134]:
# 5. 완성! model로 맘대로 할 수 있다.
# similar값이 높게 나오지 않는다....

model = gensim.models.Word2Vec.load('result_k.model')

model.most_similar("왕/Noun")
model.most_similar(positive=["형제/Noun", "여자/Noun"], negative=["남자/Noun"], topn=5)
model.most_similar(positive=["갤럭시/Noun","애플/Noun"], negative=["삼성/Noun"], topn=5)

[('매킨토시/Noun', 0.48226165771484375),
 ('ios/Alpha', 0.46322768926620483),
 ('아이폰/Noun', 0.45036351680755615),
 ('버전/Noun', 0.44701114296913147),
 ('애플사/Noun', 0.4437728524208069)]